## Collaborative Filtering

Integrantes
* Santiago
* Jose Reyes 142207
* Patricia
* Yedam Fortiz 119523

### Carga de paqueteria

In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma
import random as r
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [2]:
r.seed(1234)

### Carga de Informacion

In [3]:
links = pd.read_csv("links_small.csv")
metadata = pd.read_csv("movies_metadata.csv")
ratings = pd.read_csv("ratings_small.csv")
ratings = ratings.drop("timestamp",axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
keys_metadata={}
k=0
for i,j,z,w in zip(metadata.id,metadata.original_title,metadata.overview,metadata.release_date):
    
    try:
        keys_metadata[int(i)]=[j,z,w]
    except: 
        print("Error en ",k)
    k+=1
        

Error en  19730
Error en  29503
Error en  35587


### Por ahora consideraremos únicamente 1500 lineas de ratings, no los 100,000 registros ya que el algoritmo no está optimizado para una base de datos tan grande

In [6]:
ratings_wide = ratings.pivot_table(index = 'userId',
                                   columns = 'movieId',
                                   values = 'rating')

In [7]:
#ratings_wide,X_test = train_test_split(ratings_wide.head(1500),test_size=0.3,random_state=1234)

In [8]:
ratings_wide=ratings_wide.iloc[0:50,:]

### Creacion de matrices

In [9]:
links_values = np.array(links)
links_features = np.array(links.columns)
ratings_values = np.array(ratings_wide)
ratings_features = np.array(ratings_wide.columns)

### Funcion de costo

$$ \large J(X)  =  \frac{1}{2} \sum_{(a,i \in D)} (Y_{ai} - [UV^T]_{ai} )^2 + \frac{\lambda}{2} \sum_{a=1}^n\sum_{j=1}^k U_{aj}^2 + \frac{\lambda}{2} \sum_{i=1}^m\sum_{j=1}^k V_{ij}^2  $$

* Seleccionamos $V$ al azar y la dejamos fija y optimizamos con respecto a $U$
* Una vez actualizada la $U$, la dejamos fija y optimizamos con respecto a $V$
* Repetimos hasta que converja (variaciones entre las estimaciones de los vectores es pequeña) (optimo local)

#### Minimizacion Alternada

In [10]:
def gradiente_U (Y,U,V,k,lambda_):
    
    if k!=1:
        U_V = U.dot(V.T)
    else:
        U_V = np.outer(U,V)
    
    na = np.isnan(Y)
    
    #Tratamiento especial NA
    #gradiente = -dot_na(Y-U_V, V) + lambda_*U
    gradiente = -(Y-U_V)@V + lambda_*U
    
    return gradiente

In [11]:
def gradiente_V (Y,U,V,k,lambda_):
    
    if k!=1:
        U_V = U.dot(V.T)
    else:
        U_V = np.outer(U,V)
    
    na = np.isnan(Y)
    
    #Tratamiento especial NA
    #gradiente = -dot_na((Y-U_V).T, U) + lambda_*V
    gradiente = -(Y-U_V).T@U + lambda_*V
    
    return gradiente

In [12]:
def dot_na(X,Y):
    
    n,m = X.shape
    lista = []
    
    for i in range(n):
        pos_na = ~np.isnan(X[i,:])
        lista.append(X[i,pos_na]@Y[pos_na])
        lista_ = np.array(lista)
    
    return lista_ 

In [13]:
def descenso_gradiente_U(Y,U,V,k,lambda_,eta,epsilon,maxiter=5000):
    
    U_ = U
    i = 0 
    while True:
        U_aux = U_
        gradiente_u = gradiente_U(Y,U_,V,k,lambda_)
        U_ = U_ - eta * gradiente_u
        
        if (np.linalg.norm(U_aux-U_))< epsilon:
            break
        if i>maxiter:
            break
        i+=1

    return U_

In [14]:
def descenso_gradiente_V(Y,U,V,k,lambda_,eta,epsilon,maxiter=5000):
    
    V_ = V
    i = 0 
    while True:
        V_aux = V_
        gradiente_v = gradiente_V(Y,U,V_,k,lambda_)
        V_ = V_ - eta * gradiente_v
        
        if (np.linalg.norm(V_aux-V_))< epsilon:
            break
        if i>maxiter:
            break
        i+=1

    return V_

In [15]:
def minimizacion_alternada(Y,k,lambda_,eta,epsilon):
    """
    Objetivo:
    Realizar minimizacion alternada
    
    Insumo:
    Y - Matriz a evaluar
    k - Hiperparametro para obtimizar funcion de costo
    lambda - Hiperparametro de regularizacion
    eta - Tamaño de paso
    epsilon - Criterio de paro
    
    Resultado:
    U - Sentimiento general de cada usuario hacia las peliculas
    V - Como cada una de las peliculas es percibidas por los usuarios
    
    """
    
    n,m = Y.shape
    
    Y = np.nan_to_num(Y)
    
    U = np.random.uniform(low = 0,high = (1/np.sqrt(k)),size = [n,k])
    V = np.random.uniform(low = 0,high = (1/np.sqrt(k)),size = [m,k])
    
    #Optimizar U
    U_final = descenso_gradiente_U(Y,U,V,k,lambda_,eta,epsilon)
    
    #Optimizar V
    V_final = descenso_gradiente_V(Y,U_final,V,k,lambda_,eta,epsilon)
    
    return U_final,V_final

Podemos hacer la revisión con esta matriz

In [16]:
Y = np.array([[5, np.nan, 7], 
              [1, 1, np.nan]])

In [17]:
Y

array([[ 5., nan,  7.],
       [ 1.,  1., nan]])

In [18]:
U_Y,V_Y = minimizacion_alternada(Y,k=2,lambda_=0.1,eta=0.01,epsilon=1e-3)

In [19]:
print(Y)
print(U_Y@V_Y.T)
print(np.linalg.norm(np.nan_to_num(Y)-U_Y@V_Y.T))

[[ 5. nan  7.]
 [ 1.  1. nan]]
[[5.01690374 0.08587532 6.90413505]
 [0.8271082  0.39407857 0.60134786]]
0.8806259159622467


### En estos momentos no se consideró el procedimiento de tratamiento especial de los Nulos, ya que el programa tarda mucho tiempo en correr. Por ahora estamos considerando los registros nulos como 0.

Haremos la estimación para la información de las películas

## Selección de parámetros 

Se realiza un grid search para encontrar los mejores valores de los parámetros, entre ellos, el valor de k, tomando como criterio referencia el error NDCG: 

### Verificaremos el valor del ndcg score para diferentes valores de k y los otros parametros

In [20]:
parametros= { 'k':[3,4,5],
            'eta':[0.0001],
            'lambda':[0.25,0.5,1,1.25]}

In [21]:
def grid_search(params):
    """
    Objetivo:
    Búsqueda de hiperparámetros del modelo
    
    Insumo:
    params - Dicciionario con hyperparámetros k,eta y lambda a probar
    
    Resultado:
    Valores de k, eta y lambda con menor costo por NDCG
    
    """
    
    values=[]
    for k in params['k']:
        for eta in params['eta']:
            for lambd in params['lambda']:
                try:
                    U,V = minimizacion_alternada(ratings_wide,k=k,lambda_=lambd,eta=eta,epsilon=1e-3)
                    ndcg= ndcg_score(np.nan_to_num(ratings_wide), U@V.T)
                    values.append([k,eta,lambd,ndcg])
                    print(k,eta,lambd,ndcg)
                except:
                    print("error")
                    ndcg=np.nan
                    values.append([k,eta,lambd,ndcg])
    parameters_selection= pd.DataFrame(values, columns=['k','eta','lambda','ndcg'])
    print('best combination of parameters:')
    print(parameters_selection.loc[parameters_selection['ndcg'] == parameters_selection['ndcg'].min()])
    return pd.DataFrame(values, columns=['k','eta','lambda','ndcg'])
        

In [22]:
parameters_selection= grid_search(parametros)

3 0.0001 0.25 0.581083295001394
3 0.0001 0.5 0.5854400345780428
3 0.0001 1 0.5814598526894674
3 0.0001 1.25 0.580106781930808
3 0.0001 1.5 0.5814740426062657
3 0.0001 2 0.581148071417714
3 0.0001 5 0.5804905759516545
4 0.0001 0.25 0.586110034680311
4 0.0001 0.5 0.5852421597144636
4 0.0001 1 0.586922606182513
4 0.0001 1.25 0.5866517896872719
4 0.0001 1.5 0.5869135682546213
4 0.0001 2 0.5881671799900656
4 0.0001 5 0.5833557860769437
5 0.0001 0.25 0.5894824724601825
5 0.0001 0.5 0.5901410478817646
5 0.0001 1 0.5919037748481795
5 0.0001 1.25 0.5905153700920902
5 0.0001 1.5 0.591773574448952
5 0.0001 2 0.5875080021684704
5 0.0001 5 0.5872270366250905
6 0.0001 0.25 0.5937193900354366
6 0.0001 0.5 0.5919858662997757
6 0.0001 1 0.5991837847224761
6 0.0001 1.25 0.597834714319735
6 0.0001 1.5 0.5976959600622527
6 0.0001 2 0.6015218871434439
6 0.0001 5 0.5931140974400946
7 0.0001 0.25 0.617533965750197
7 0.0001 0.5 0.6144023199985317
7 0.0001 1 0.6054670947593479
7 0.0001 1.25 0.604784918154467
7

## Entrenamos el modelo con los mejores parámetros encontrados:

In [27]:
U_final,V_final = minimizacion_alternada(ratings_wide,k=3,lambda_=1.25,eta=0.0001,epsilon=1e-3)

### 5 Mejores recomendaciones

Dada que con el modelo es posible construir una matríz de calificaciones de películas completo, es posible construir el sistema de recomendación. 

Sea $R=UV^T$ la matríz de calificaciones completa. En este caso cada reglon representa un usuario y cada columna una película. El valor entrada de la matríz representa una calificación potencial. En el sistema de recomendación se selecciona a un usuario, se filtran las películas que ya ha visto, y de las restantes se recomiendan las 5 películas con mejor calificación.

In [28]:
recomendation_matrix = U_final@V_final.T

def recomendacion(user,recomendation_matrix,ratings_features,ratings_values):
    """
    Objetivo:
    Recomendar películas dada una matríz de recomendación
    
    Insumo:
    user - Número de usuario
    recomendation_matrix - Matríz de recomendación
    ratings_features - Array con id de Películas
    ratings_valuess - Matríz rala con calificaciones 
    
    Resultado:
    Lista con 5 películas que el usuario no ha vista y 
    que tienen mejor calificación para él. 
    
    """
    
    rec_user = recomendation_matrix[user,]
    ratings_user = ratings_values[user,]
    
    not_watched = pd.isnull([x for x in ratings_values[user,]])
    rec_user[np.invert(not_watched)]=np.array([-np.inf]*sum(np.invert(not_watched))) 
    
    idx_top5=rec_user.argsort()[-5:][::-1]
    #print("Calificaciones: ",rec_user[idx_top5])
    print("Puedes ver las películas: ",ratings_features[idx_top5])
    return list(ratings_features[idx_top5])

In [39]:
def Recomendar():
    """
    Objetivo:
    Recomendar una película a un usuario a determinar
    
    Insumo:
    input - Número de usuario 
    
    Funciones/Datos Anidados:
    Función recomendación
    Diccionario que asocia id de película con título, año de lanzamiento y sinópsis.
    
    Resultado:
    5 películas recomendadas por la función recomendación,
    su título, año de lanzamiento y sinópsis. 
    
    
    """
    
    user = int(input("Número de Usuario :"))
    try:
        movies=recomendacion(user,recomendation_matrix,ratings_features,ratings_values)
        print("_____________________________")
        for i in movies:
            try:
                movie_info=keys_metadata[i]
                print("Título: ",movie_info[0])
                print("Fecha de Lanzamiento: ",movie_info[2])
                print("Sinópsis: ",movie_info[1])
                print("_____________________________")
            except:
                print("No tenemos información disponible de la película :",i)
                print("_____________________________")
    except:
        print("No tenemos registro de ese número de usuario")


In [45]:
Recomendar()

Número de Usuario :20
Puedes ver las películas:  [2571   50 2959 1997  110]
_____________________________
No tenemos información disponible de la película : 2571
_____________________________
No tenemos información disponible de la película : 50
_____________________________
Título:  License to Wed
Fecha de Lanzamiento:  2007-07-04
Sinópsis:  Newly engaged, Ben and Sadie can't wait to start their life together and live happily ever after. However Sadie's family church's Reverend Frank won't bless their union until they pass his patented, "foolproof" marriage prep course consisting of outrageous classes, outlandish homework assignments and some outright invasion of privacy.
_____________________________
Título:  Deux frères
Fecha de Lanzamiento:  2004-04-07
Sinópsis:  Two tigers are separated as cubs and taken into captivity, only to be reunited years later as enemies by an explorer (Pearce) who inadvertently forces them to fight each other.
_____________________________
Título:  Trois 